In [1]:
import os
import sys
from pathlib import Path

import asyncio
import nest_asyncio
from playwright.async_api import async_playwright

sys.path.append(str(Path.cwd().parents[0]))
from src.VRCTwitterImageLoader.scripts.image_url import get_tweet_embedcode

In [2]:
import os
from playwright.async_api import async_playwright
import asyncio

async def save_html_as_png(converted_urls, file_name="tweet.html"):
    """
    URLリストから埋め込みツイートをスクリーンショットとして保存する非同期関数。
    
    各ツイートの埋め込みHTMLコードを取得し、ローカルのHTMLファイルに書き込みます。
    次に、Playwrightの非同期APIを使用してそのHTMLファイルをヘッドレスモードのChromiumブラウザで開き、
    ツイート要素が完全に読み込まれるのを待ってから、その要素のスクリーンショットをPNG形式で保存します。

    Args:
        converted_urls (list): 埋め込みツイートHTMLを取得するためのURLのリスト。
        file_name (str): 埋め込みツイートHTMLを保存するローカルファイルの名前。
                        デフォルトは"tweet.html"です。

    Returns:
        None: スクリーンショットは現在の作業ディレクトリにPNGファイルとして保存されます。
    """
    async with async_playwright() as p:
        # ヘッドレスモードのChromiumブラウザを起動
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()

        for index, render_url in enumerate(converted_urls):
            # ステップ1: ツイートの埋め込みHTMLを取得 (get_tweet_embedcodeは既存のヘルパー関数と仮定)
            html_content = get_tweet_embedcode(render_url)

            # ステップ2: 取得したHTMLコンテンツをローカルファイルに書き込む
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(html_content)

            # ステップ3: ローカルHTMLファイルを"file://" URLで開く
            local_url = "file://" + os.path.abspath(file_name)
            page = await context.new_page()
            await page.goto(local_url)

            # ステップ4: 埋め込みツイート（Twitterのblockquote要素）が読み込まれるまで待つ
            await page.wait_for_selector("blockquote.twitter-tweet")

            # ステップ5: ビューポートサイズを調整してツイートが全て表示されるようにする
            await page.set_viewport_size({"width": 550, "height": 1000})

            # ステップ6: ツイート要素のみのスクリーンショットを撮る
            element = await page.query_selector("blockquote.twitter-tweet")
            await element.screenshot(path=f"cropped_screenshot_{index}.png")

        # ブラウザを閉じる
        await browser.close()

In [3]:
# nest_asyncioを使ってイベントループをネストできるようにする
nest_asyncio.apply()

In [4]:
# asyncio.runを使用して非同期関数を実行
url = ["https://twitter.com/H5T42/status/1803982291984879796"]  # URLはリスト形式で渡す
asyncio.run(save_html_as_png(converted_urls=url))

Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Connection.run() done, defined at c:\Users\varyu\Desktop\development\VRCTwitter\VRCTwitterImageLoader\.venv\Lib\site-packages\playwright\_impl\_connection.py:265> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\varyu\AppData\Roaming\uv\python\cpython-3.11.6-windows-x86_64-none\Lib\asyncio\tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\varyu\Desktop\development\VRCTwitter\VRCTwitterImageLoader\.venv\Lib\site-packages\playwright\_impl\_connection.py", line 272, in run
    await self._transport.connect()
  File "c:\Users\varyu\Desktop\development\VRCTwitter\VRCTwitterImageLoader\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\varyu\Desktop\development\VRCTwitter\VRCTwitterImageLoader\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 120, in conne

NotImplementedError: 

In [5]:
from playwright.sync_api import sync_playwright
import os

def save_html_as_png_sync(converted_urls, file_name="tweet.html"):
    """
    URLリストから埋め込みツイートをスクリーンショットとして保存する同期版の関数。
    
    Playwrightの同期APIを使用して、HTMLファイルを開いてツイートをスクリーンショットとして保存します。
    """
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        context = browser.new_context()

        for index, render_url in enumerate(converted_urls):
            html_content = get_tweet_embedcode(render_url)
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(html_content)

            local_url = "file://" + os.path.abspath(file_name)
            page = context.new_page()
            page.goto(local_url)

            page.wait_for_selector("blockquote.twitter-tweet")
            page.set_viewport_size({"width": 550, "height": 1000})

            element = page.query_selector("blockquote.twitter-tweet")
            element.screenshot(path=f"cropped_screenshot_{index}.png")

        browser.close()

# 関数の実行
url = ["https://twitter.com/H5T42/status/1803982291984879796"]
save_html_as_png_sync(converted_urls=url)


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.